In [1]:
import tensorflow as tf

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [19]:
BAT = 16
def GENERATOR(Z):    
    c_w1 = tf.Variable(tf.random_normal([100, 256]))
    c_w2 = tf.Variable(tf.random_normal([256, 256]))
    c_w3 = tf.Variable(tf.random_normal([256, 256]))
    c_w4 = tf.Variable(tf.random_normal([256, 256]))
    c_w5 = tf.Variable(tf.random_normal([256, 100]))
    b = tf.Variable(tf.random_normal([16, 256]))


    x = tf.matmul(Z,c_w1)
    x = tf.nn.relu(x)
    x = tf.matmul(x,c_w2)
    x = tf.nn.relu(x)
    x = tf.matmul(x,c_w3)
    x = tf.nn.relu(x)
    x = tf.matmul(x,c_w4)
    x = tf.nn.relu(x)
    x = tf.matmul(x,c_w5)
    x = tf.nn.relu(x)

    w1 = tf.Variable(tf.random_normal([100, 1024 * 4 * 4]))
    x = tf.matmul(x,w1)
    re = tf.reshape(x,[-1,4,4,1024])

    filter = tf.random_normal([1, 1, 512, 1024])
    filter = tf.Variable(filter)
    re = tf.layers.batch_normalization(re)
    re = tf.nn.conv2d_transpose(re, filter, [BAT,8,8,512], [1,2,2,1], name='a')
    re = tf.nn.relu(re)

    filter = tf.random_normal([5, 5, 256, 512])
    filter = tf.Variable(filter)
    re = tf.layers.batch_normalization(re)
    re = tf.nn.conv2d_transpose(re, filter, [BAT,16,16,256], [1,2,2,1], name='a')
    re = tf.nn.relu(re)

    filter = tf.random_normal([5, 5, 128, 256])
    filter = tf.Variable(filter)
    re = tf.layers.batch_normalization(re)
    re = tf.nn.conv2d_transpose(re, filter, [BAT,32,32,128], [1,2,2,1], name='a')
    re = tf.nn.relu(re)

    filter = tf.random_normal([5, 5, 3, 128])
    filter = tf.Variable(filter)
    re = tf.nn.conv2d_transpose(re, filter, [BAT,64,64,3], [1,2,2,1], name='a')
    re = tf.nn.tanh(re)
    return re

In [27]:
def DISCRIMINATOR(X):
    W1 = tf.Variable(tf.random_normal([5,5,3,128]))
    W2 = tf.Variable(tf.random_normal([5,5,128,256]))
    W3 = tf.Variable(tf.random_normal([5,5,256,512]))
    W4 = tf.Variable(tf.random_normal([5,5,512,1024]))
    d_w1 = tf.Variable(tf.random_normal([1024 * 4 * 4, 1]))

    y = tf.nn.conv2d(X,W1, strides=[1,2,2,1], padding="SAME")
    y = tf.nn.leaky_relu(y)
    y = tf.layers.batch_normalization(y)
    y = tf.nn.conv2d(y,W2, strides=[1,2,2,1], padding="SAME")
    y = tf.nn.leaky_relu(y)
    y = tf.layers.batch_normalization(y)
    y = tf.nn.conv2d(y,W3, strides=[1,2,2,1], padding="SAME")
    y = tf.nn.leaky_relu(y)
    y = tf.layers.batch_normalization(y)
    y = tf.nn.conv2d(y,W4, strides=[1,2,2,1], padding="SAME")
    y = tf.nn.leaky_relu(y)
    y = tf.layers.flatten(y)
    y = tf.matmul(y, d_w1)
    return y

In [28]:
G = GENERATOR(Z)
D_real = DISCRIMINATOR(X)
D_gene = DISCRIMINATOR(G)

In [34]:
# BEGAN
K = tf.placeholder(tf.float32)
D_loss = D_real - K*D_gene
G_loss = D_gene

D_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(D_loss))
G_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(G_loss))

# WGAN
D_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(D_loss))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(G_loss))

In [ ]:
# BEGAN
k_curr = -1
gamma = 0.5
lam = 1e-3

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for it in range(1000000):
    

    _, D_real_curr = sess.run(
        [D_solver, D_real],
        feed_dict={X: np.array(images[:16]), Z: sample_z(16, 100), K: k_curr}
    )

    _, D_fake_curr = sess.run(
        [G_solver, D_gene],
        feed_dict={X: np.array(images[:16]), Z: sample_z(16, 100)}
    )

    k_curr = k_curr + lam * (gamma*D_real_curr - D_fake_curr)

    if it % 1 == 0:
        measure = D_real_curr + np.abs(gamma*D_real_curr - D_fake_curr)

        print('Iter-{%04d}; Convergence measure: {:%f}' % (it, measure[0]))

Iter-{0000}; Convergence measure: {:63624872.000000}
Iter-{0001}; Convergence measure: {:-36975160.000000}
Iter-{0002}; Convergence measure: {:-130154712.000000}
Iter-{0003}; Convergence measure: {:-39350672.000000}
Iter-{0004}; Convergence measure: {:-303527616.000000}


KeyboardInterrupt: 

In [28]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [29]:
train_D = tf.train.AdamOptimizer(0.001).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.001).minimize(loss_G)

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [33]:
made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})
c = 0
for j in made_image:
    c += 1
    cv2.imwrite('{i}-{c}.jpg'.format(i=1,c=c),j)

In [7]:
from keras.preprocessing.image import load_img, img_to_array
import keras
import tensorflow as tf
import numpy as np
import cv2
import os
import math
files = os.listdir(r'..\AnimeFaceDetecter\Datas\result\face')
keras.backend.set_image_data_format('channels_last')
#keras.backend.set_image_data_format('channels_first')
images = []
for i in files:
    image = load_img(r'..\AnimeFaceDetecter\Datas\result\face\{}'.format(i),False,target_size=(64,64))
    image = img_to_array(image)
    image = image / 256
    images.append(image)
print(image.shape)

Using TensorFlow backend.


(64, 64, 3)


In [8]:
def sample_z(m, n):
    return np.random.uniform(-2., 2., size=[m, n])

In [36]:
for i in range(100):
    noise = sample_z(16,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:np.array(images[:16]),
                      Z: noise})
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:np.array(images[:16]),
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))
    if i % 20 == 0:
        made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})
        c = 0
        for j in made_image:
            c += 1
            cv2.imwrite('{i}-{c}.jpg'.format(i=i,c=c),j * 256)

0   D : 9573849.0	 G : 59432440.0
1   D : 12018224.0	 G : 54832116.0


KeyboardInterrupt: 

In [42]:
c_w1 = tf.Variable(tf.random_normal([100, 256]))
b = tf.Variable(tf.random_normal([16, 256]))

In [43]:
x = tf.matmul(Z,c_w1) + b

In [44]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(x, feed_dict={Z:sample_z(16,100)})

array([[  6.0903063, -10.780539 , -10.950874 , ...,   5.6719384,
         -3.0936234,  12.862015 ],
       [ 12.690213 ,  13.570845 ,  -7.999601 , ...,   1.6188432,
         -3.9302454, -17.548717 ],
       [  6.1085987,   3.1240082,  13.878397 , ..., -20.01085  ,
         -2.0349321,  12.512978 ],
       ...,
       [ -5.2602334,  10.730547 ,  -4.6213646, ...,   2.3149886,
         10.564762 ,   8.231128 ],
       [ -5.391034 , -11.055565 ,  -1.5542789, ..., -12.53598  ,
         -9.031625 ,  -4.982078 ],
       [ -2.850051 ,  -7.8083634,  24.759752 , ...,  12.285364 ,
         -4.4942055,   9.447315 ]], dtype=float32)

(16, 1)

(16, 10)